# FCN score for cycleGAN models

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from evaluate.networks import fcn_8s
from skimage.io import imread
import torchvision.transforms as tvt
import PIL
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from FCN.networks import FCN8s, VGGNet
from FCN.citydataset import city, classes_city, train_transforms

from evaluate.metrics import FCNScore
from evaluate import losses

## Dataset

### Folder

In [ ]:
folder = r"./datasets/cyclegan/"
mode = r"test"

### Ground Truth FCN score

In [ ]:
batch_size =1
city_dataset = city(mode=mode,classes=classes_city)
city_loader  = DataLoader(dataset=city_dataset, batch_size=batch_size)

### Model

In [ ]:
class FCNScore(torch.nn.Module):
    def __init__(self, model=None, num_classes=21):
        super(FCNScore, self).__init__()
        self.model = model
        self.name = 'fcn'
        self.num_classes = num_classes

    def forward(self, input, target):
        self.model.eval()
        with torch.no_grad():
            output = self.model(input.cuda())
            labels = output.softmax(dim=1).argmax(dim=1)
        return losses.label_score(labels,target,self.num_classes)

In [ ]:
fcn = FCN8s(pretrained_net=VGGNet(requires_grad=False),n_class = len(classes_city)).cuda()
# change path to the correct path
fcn.load_state_dict(torch.load(r"FCN/checkpoints/EXP_11/checkpoint.pth.tar"))

# CityScapes : FCN/checkpoints/EXP_11/checkpoint.pth.tar
# Maps : CN/checkpoints/EXP_16/checkpoint.pth.tar"

In [ ]:
fcn_score = FCNScore(model = fcn, num_classes=len(classes_city))

In [ ]:
metrics = []
for i, batch in enumerate(city_loader):
    ppa, pca, iou_acc =   fcn_score(batch['image'], batch['target'])
    metrics.append(batch_size*np.array([ppa, pca, iou_acc]))
metrics = np.array(metrics)

In [ ]:
metrics.mean(axis = 0)

In [ ]:
metrics.std(axis = 0)

### Cycle Gan FCN score

In [ ]:
DF = dict()

In [ ]:
# can be changed to the different paths
folder = r"results/{}/test_latest/images"
model = r"horses_cyclegan_sn"

In [ ]:
class CycleCity(Dataset):
    def __init__(self, model, folder=folder, transforms=train_transforms, classes=classes_city):
        super(CycleCity, self).__init__()
        self.data_path = folder.format(model)
        self.transforms = transforms
        self.ims = list(filter(lambda x: 'fake_A' in x, os.listdir(folder.format(model))))
        self.masks = list(map(lambda x: x.replace('fake_A', 'real_B'), self.ims))
        self.classes = classes
    def __len__(self):
        return len(self.ims)
    def make_mask(self, mask):
        def find_cluster(vec, classes=self.classes):
            rscores = np.zeros((256 * 256, len(classes)))
            for i in range(len(classes)):
                rscores[:, i] = np.linalg.norm(vec - np.repeat(classes[i].reshape(1, 3), 256 * 256, axis=0), axis=1)
            vc = np.argmin(rscores, axis=1)
            return vc

        def find_cluster_torch(vec, classes=self.classes):
            rscores = torch.zeros((256 * 256, len(classes)))
            for i in range(len(classes)):
                rscores[:, i] = torch.norm(
                    torch.cuda.FloatTensor(vec.reshape(-1, 3)) - torch.cuda.FloatTensor(
                        classes[i].reshape(1, 3)).repeat(256 * 256, 1),
                    dim=1
                )

            vc = rscores.argmin(dim=1)
            return vc
        clusters = find_cluster_torch(mask.reshape(-1,3))
        mask = clusters.view(256,256).type(torch.LongTensor)
        return mask
    def __getitem__(self, index):
        I, M = imread(os.path.join(self.data_path,self.ims[index])), imread(os.path.join(self.data_path,self.masks[index]))
        mask = self.make_mask(M)
        if self.transforms is not None:
            I = self.transforms(I)
        return {'image':I, 'target':mask, 'im':self.ims[index]}

In [ ]:
batch_size = 1
cycle_city = CycleCity(model=model)
cycle_city_loader = DataLoader(dataset=cycle_city, batch_size=batch_size)

In [ ]:
cycle_metrics = []
ims = []
for i, batch in enumerate(cycle_city_loader):
    ppa, pca, iou_acc =   fcn_score(batch['image'], batch['target'])
    ims.append(batch['im'])
    cycle_metrics.append(batch_size*np.array([ppa, pca, iou_acc]))
cycle_metrics = np.array(cycle_metrics)

In [ ]:
print(cycle_metrics.sum(axis = 0)/len(cycle_city))
print(cycle_metrics.std(axis = 0))

In [ ]:
data = list(filter(lambda x: '14'== x[0][0][:2], zip(ims, cycle_metrics)))
D = list(map(lambda x: [x[0][0], x[1][0], x[1][1], x[1][2]], data))
DF[model] = pd.DataFrame(D).sort_values(by=0)

In [ ]:
# change the model name to the one you want to evaluate

def imbyim(DF):
    df = dict()
    for im in DF['horses_cyclegan'].keys():
        values = np.zeros(4)
        for key in DF:
            if 'pretrained' in key:
                print(DF[key][DF[key]["0"] == im][1])
                values[0] = DF[key][DF[key][0] == im][1]
            elif 'sn' in key:
                values[3] = DF[key][DF[key][0] == im][1]
            elif 'wgan' in key:
                values[2] = DF[key][DF[key][0] == im][1]
            else:
                values[1] = DF[key][DF[key][0] == im][1]
            df[im] = pd.DataFrame(values, columns = ['PreTrained', 'Baseline', 'WGANGP', 'SN'])
    return df

In [ ]:
D = pd.DataFrame(np.hstack((DF[key][1].values.reshape(-1,1) for key in DF)).T, columns = DF['cityscapes_label2photo_pretrained'][0].values)
D.index = ['PreTrained', 'Baseline', 'WGANGP', 'SN']

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

In [ ]:
D.style.apply(highlight_max)